In [1]:
using Pkg
Pkg.activate(".")

 Activating environment at `C:\Users\adria\Dropbox\Projects\IrisClustering\Project.toml`


In [3]:
using Revise

using Genie, Genie.Router
using Genie.Renderer.Html

using Stipple, Stipple.Layout, Stipple.Elements
using StippleUI, StippleUI.Select, StippleUI.Table, StippleUI.Range
using StippleCharts, StippleCharts.Charts

using CSV, DataFrames, Clustering

import Genie.Renderer.Html: select

data = DataFrames.insertcols!(DataFrame!(CSV.File("data/iris.csv"))[:, 2:end], :Cluster => zeros(Int, 150))

Base.@kwdef mutable struct Model <: ReactiveModel
  iris_data::R{DataTable} = DataTable(data)
  credit_data_pagination::DataTablePagination = DataTablePagination(rows_per_page=50)

  plot_options::PlotOptions = PlotOptions(chart_type=:scatter, xaxis_type=:numeric)
  iris_plot_data::R{Vector{PlotSeries}} = PlotSeries[]
  cluster_plot_data::R{Vector{PlotSeries}} = PlotSeries[]

  features::R{Vector{String}} = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]
  xfeature::R{String} = ""
  yfeature::R{String} = ""

  no_of_clusters::R{Int} = 3
  no_of_iterations::R{Int} = 10
  clustering_features::R{Vector{String}} = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]
end

Stipple.register_components(Model, StippleCharts.COMPONENTS)

const model = Stipple.init(Model())

function plot_data(cluster_column::Symbol)
  result = Vector{PlotSeries}()
  isempty(model.xfeature[]) || isempty(model.yfeature[]) && return result

  dimensions = Dict()
  for s in Array(data[:, cluster_column]) |> unique!
    dimensions[s] = []

    for r in eachrow(data[data[cluster_column] .== s, :])
      push!(dimensions[s], [r[Symbol(model.xfeature[])], r[Symbol(model.yfeature[])]])
    end

    push!(result, PlotSeries("$s", PlotData(dimensions[s])))
  end

  result
end

function compute_clusters!()
  features = collect(Matrix(data[:, [Symbol(c) for c in model.clustering_features[]]])')
  result = kmeans(features, model.no_of_clusters[]; maxiter=model.no_of_iterations[])
  data[:Cluster] = assignments(result)
  model.iris_data[] = DataTable(data)
  model.cluster_plot_data[] = plot_data(:Cluster)

  nothing
end

onany(model.xfeature, model.yfeature) do (_...)
  model.iris_plot_data[] = plot_data(:Species)
  compute_clusters!()
end

onany(model.no_of_clusters, model.no_of_iterations, model.clustering_features) do (_...)
  compute_clusters!()
end

function ui(model::Model)
  layout([
    page(root(model), class="container", [
      h3("Iris data k-means clustering")
      row([
        cell(size=10, [
          h5("Iris data")
          table(@data(:iris_data); pagination=:credit_data_pagination,
                                    dense=true, flat=true, style="height: 350px;")
        ])
        cell(size=2, [
          h5("Clustering")
          row([
            cell([
              h6("Number of clusters")
              slider(@data(:no_of_clusters), 1:1:20;
                      markers=true, label=true)
            ])
            cell([
              h6("Number of iterations")
              slider(@data(:no_of_iterations), 10:10:200;
                      markers=true, label=true)
            ])
            cell([
              h6("Features")
              select(:clustering_features;
                      options=:features, multiple=true)
            ])
          ])
        ])
      ])
      row([
        cell(size=5, [
          h5("Species clusters")
          plot(@data(:iris_plot_data); options=:plot_options)
        ])
        cell(size=5, [
          h5("k-means clusters")
          plot(@data(:cluster_plot_data); options=:plot_options)
        ])
        cell(size=2, [
          h5("Plotting")
          row([
            cell([
              h6("X feature")
              select(:xfeature; options=:features)
            ])
          ])
          row([
            cell([
              h6("Y feature")
              select(:yfeature; options=:features)
            ])
          ])
        ])
      ])
    ])

    style("
    h1,h2,h3,h4,h5,h6 {
      margin-bottom: auto;
      margin-top: auto;
    }
    .container {
      padding: 10px;
    }
    ")
  ],
  title="Iris Data K-Means Clustering"
  )
end

route("/") do
  ui(model) |> html
end

up()


Web Sockets server running at 127.0.0.1:8001 



Web Server starting at http://127.0.0.1:8000 

Web Server running at http://127.0.0.1:8000 


Genie.AppServer.ServersCollection(Task (runnable) @0x0000000035a2ce70, Task (runnable) @0x00000000353c7990)

┌ Info: / 200
└ @ Genie.Router C:\Users\adria\.julia\dev\Genie\src\Router.jl:161
┌ Info: /stipple.js?v=0.9894484634464353 200
└ @ Genie.Router C:\Users\adria\.julia\dev\Genie\src\Router.jl:161
┌ Info: /js/stipple/vue.js 200
└ @ Genie.Router C:\Users\adria\.julia\dev\Genie\src\Router.jl:161
┌ Info: /js/stipple/underscore-min.js 200
└ @ Genie.Router C:\Users\adria\.julia\dev\Genie\src\Router.jl:161
┌ Info: /__/channels.js?v=1.0.0 200
└ @ Genie.Router C:\Users\adria\.julia\dev\Genie\src\Router.jl:161
┌ Info: /css/stipple/quasar.min.css 200
└ @ Genie.Router C:\Users\adria\.julia\dev\Genie\src\Router.jl:161
┌ Info: /css/stipple/stipple.css 200
└ @ Genie.Router C:\Users\adria\.julia\dev\Genie\src\Router.jl:161
┌ Info: /js/stipple/quasar.umd.min.js 200
└ @ Genie.Router C:\Users\adria\.julia\dev\Genie\src\Router.jl:161
┌ Info: /js/stipple/apexcharts.min.js 200
└ @ Genie.Router C:\Users\adria\.julia\dev\Genie\src\Router.jl:161
┌ Info: /js/stipple/vue-apexcharts.min.js 200
└ @ Genie.Router C:\Us